# Exercise 3 · Missing-Data Case Studies  
*Notebook Overview & Road-Map*

Welcome to **Exercise 3**, a two-part, eight-case sprint through real-world missing-data puzzles inspired by the ABCD Study.  
You’ll practice *detecting*, *classifying*, and *cleaning* a variety of missingness patterns—then see how each choice changes your results.

## 🎯 Overall Learning Objectives

1. **Identify & classify structured missing-value codes**  
   Locate `777 Refused`, `999 Don’t-know`, sentinel blanks, and branch-skip cells and explain what each signals.

2. **Differentiate MCAR / MAR / MNAR scenarios**  
   Use skip patterns, non-response codes, and quick EDA to defend a mechanism diagnosis.

3. **Select & justify handling strategies**  
   Recommend (and code) a simple but appropriate remedy then quantify how a naïve vs. cleaned approach shifts key statistics.

---

## 🔹 Part 1 ( Case 1 – 4 ) — **Mechanisms in Miniature**  
Four bite-sized toy surveys walk you through the **three canonical mechanisms**:

| Case | Scenario (1-sentence) | Likely Mechanism | Skill Focus |
|------|-----------------------|------------------|-------------|
| 1 | A single row randomly corrupted | **MCAR** | Harmless deletion vs. mean-fill |
| 2 | High-ADHD teens quit late items | **MAR** | Imputation choices & bias |
| 3 | High-risk youth drop 6-mo follow-up | **MNAR (or MAR)** | Conservative fixes & sensitivity |
| 4 | High-stress students skip depression item | **MAR → regression** | Using strong predictors to impute |

---

## 🔹 Part 2 ( Case 5 – 8 ) — **ABCD-Style Cases**

| Case | Instrument & Twist | Special Codes to Untangle |
|------|-------------------|---------------------------|
| 5 | **Youth Cigarette Expectancies** — item added mid-wave | Wave-design blanks + `777` Decline |
| 6 | **Parental Rules (Alcohol & Vaping)** — new ENDS branch | Wave blanks · branch blanks · `777` |
| 7 | **Household Marijuana-Smoke Exposure** — numeric follow-ups | Blank vs DK-flag `999` |
| 8 | **School Attendance & Grades** — IEP branch logic | `444` Not-Applicable · `777` Decline |

Each Part 2 notebook follows an identical 5-step path: **Setup ▸ Codebook Audit ▸ Visual Diagnostics ▸ Naïve vs Clean Bias Check ▸ Reflection**.

### Folder Structure  
Download the **Exercise3.zip** from Canvas; you’ll find:  
- `Exercise3.ipynb` – The working notebook with Parts 1 & 2  
-  synthetic_su_y_cigexp_sample.csv # Case 5
-  synthetic_su_p_rule_sample.csv # Case 6
-  synthetic_mj_smoke_sample.csv # Case 7
-  synthetic_fc_p_sag_sample.csv # Case 8

### Collaboration & Submission  
1. **Group Work:**  
   - Open the shared **Exercise 3 PowerPoint** in Canvas Collaborations  
   - As a team, complete the slides corresponding to each Part 1 case and Part 2 hunt  
2. **Individual Submission:**  
   - Once your group has finalized the shared PPT, **download your own copy**  
   - Submit that PowerPoint under **Canvas › Assignments › Exercise 3**  


## Part 1 – Missing Mechanism Case Studies  
In the first half of **E3** you’ll work through four bite-sized, self-contained surveys that illustrate the three canonical missing-data mechanisms:

| Case | Nick-name & Scenario | Likely Mechanism | Key Learning Goal |
|------|----------------------|------------------|-------------------|
| 1 | **“The Lost Survey”** – one entire respondent’s row is corrupted | MCAR | Spot purely random loss and gauge harmless remedies |
| 2 | **“Survey Fatigue Study”** – high-ADHD teens skip the last 3 items | MAR | Trace missingness back to an observed trait and compare imputation styles |
| 3 | **“Treatment Drop-Out Study”** – high-risk youth disappear at follow-up | MNAR (or MAR) | Debate whether relapse-related non-response is ignorable and test conservative fixes |
| 4 | **“Mental-Health Screening”** – high-stress students skip the depression item | MAR → regression | Exploit strong predictors to impute a sensitive outcome |

For each case you will:

1. **Diagnose** the mechanism using quick EDA.  
2. **Choose** one ad-hoc remedy and apply it.  
3. **Reflect** on how the remedy alters sample size, means, and variances.

### Team Assignment Table

| Group Member | Lead Case Study |
|--------------|-----------------|
| **Member 1** | Case 1 – The Lost Survey (MCAR) |
| **Member 2** | Case 2 – Survey Fatigue (MAR) |
| **Member 3** | Case 3 – Treatment Drop-Out (MNAR/MAR) |
| **Member 4** | Case 4 – Mental-Health Screen (MAR → Reg Impute) |

*If your group has three people then simply skip a case study.*  


## Case 1 – “The Lost Survey”  *(MCAR Candidate)*  
A REDCap export glitch corrupted one participant’s questionnaire file.  
Only the *fourth* row of data is affected; all other rows are intact.  
Your job:

1. **Explore** the pattern of missing values.  
2. **Diagnose** the likely missingness mechanism.  
3. **Apply** one simple remedy and comment on its impact.

> *Hint:* The glitch was random—any participant’s row could have been lost.


In [ ]:
# Case 1: set-up & quick EDA
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

case1 = pd.DataFrame({
    "q_anxiety":[1, 2, 3, np.nan],
    "q_stress" :[2, 2, 4, np.nan],
    "q_sleep"  :[7, 6, 5, np.nan]
})

display(case1)

# basic summary
print("\nMissing count by column:")
print(case1.isna().sum())

print("\nMissing count by row:")
print(case1.isna().sum(axis=1))

# Visualize missing data pattern
print("\nMissing Data Pattern:")
plt.figure(figsize=(8, 4))
sns.heatmap(case1.isna(), 
            yticklabels=["Row 1", "Row 2", "Row 3", "Row 4"],
            cbar_kws={'label': 'Missing Data'},
            cmap=['lightblue', 'red'])
plt.title("Missing Values Heatmap\n(Red = Missing, Blue = Present)")
plt.xlabel("Survey Questions")
plt.ylabel("Participants")
plt.tight_layout()
plt.show()

# Alternative simple visualization
print("\nSimple missing pattern check:")
missing_pattern = case1.isna().astype(int)
print(missing_pattern)

### 1️⃣ Diagnose the Missingness Mechanism  
*Use the space below to jot down your reasoning.*

- **Observed pattern:** Only row 4 is missing *every* value.  
- **Likely mechanism (MCAR / MAR / MNAR):**  
- **Why?**  


### 2️⃣ Choose & Apply a Remedy  
Pick **one** of the three ad-hoc strategies:

| Option | When It Makes Sense | One-liner Hint |
|--------|---------------------|----------------|
| **A. Listwise Deletion** | MCAR & small data loss acceptable | `clean = case1.dropna()` |
| **B. LOCF** | Longitudinal data missing a time-point | `clean = case1.ffill()` |
| **C. Mean / Median Imputation** | Small % missing, symmetric distrib. | `clean = case1.fillna(case1.mean())` |

> **Task:** Choose the approach that best fits your diagnosed mechanism and evaluate its assumptions.

In [ ]:
# --- Remedy code cell ---
# ⬇️ Uncomment the approach you decide to test.

# ----- Option A: Listwise Deletion -----
# clean = case1.dropna()

# ----- Option B: LOCF (not ideal here) -----
# clean = case1.ffill()

# ----- Option C: Mean Imputation -----
# clean = case1.fillna(case1.mean())

print("Cleaned data:")
display(clean)

print("\n" + "="*50)
print("📊 BEFORE vs AFTER COMPARISON")
print("="*50)

print("BEFORE cleaning:")
print(f"Sample size: {len(case1)} participants")
print(f"Complete responses: {case1.dropna().shape[0]} participants")
print(f"Missing values: {case1.isna().sum().sum()} total")

print("\nBEFORE - Descriptive statistics (complete cases only):")
print(case1.describe().round(2))

print("\n" + "-"*30)
print("AFTER cleaning:")
print(f"Sample size: {len(clean)} participants")
print(f"Complete responses: {clean.dropna().shape[0]} participants") 
print(f"Missing values: {clean.isna().sum().sum()} total")

print("\nAFTER - Descriptive statistics:")
print(clean.describe().round(2))

print("\n" + "-"*30)
print("🔍 KEY CHANGES:")
print(f"Rows lost/gained: {len(clean) - len(case1.dropna())}")
print(f"Values added: {clean.count().sum() - case1.count().sum()}")

# Compare means for each variable
print("\nMean comparison:")
for col in case1.columns:
    before_mean = case1[col].mean()
    after_mean = clean[col].mean() 
    change = after_mean - before_mean
    print(f"{col:12}: {before_mean:5.2f} → {after_mean:5.2f} (Δ={change:+5.2f})")

# Compare standard deviations  
print("\nStandard deviation comparison:")
for col in case1.columns:
    before_std = case1[col].std()
    after_std = clean[col].std()
    change = after_std - before_std if pd.notna(before_std) else np.nan
    print(f"{col:12}: {before_std:5.2f} → {after_std:5.2f} (Δ={change:+5.2f})")

### 3️⃣ Reflection – How Did the Remedy Change Your Data?  
#### **Answer Reflection Questions in Working Table 1 in the Group Document**

**Use the comparison output above to answer:**

- **Method used:** [A/B/C]
- **Rows/values removed or altered:** *(Look at "Sample size" and "Values added" from output)*
- **Mean & variance differences:** *(Check the "Mean comparison" and "Standard deviation comparison" sections)*  
- **Would your substantive conclusions change?** *(Consider: Are the changes large enough to matter for your research question?)*

**Write 3-4 sentences summarizing:**
1. What specific changes occurred to your dataset?
2. Whether these changes would affect conclusions about anxiety, stress, and sleep relationships?
3. Why your chosen method was/wasn't appropriate for this scenario?

## Case 2 – "The Survey Fatigue Study"  *(MAR Candidate)*  
A research team is studying adolescent attitudes toward substance use using a comprehensive 10-question survey. During data collection, they notice a concerning pattern: participants with higher **ADHD symptoms** (measured on the DSM-aligned 0-18 scale) tend to have incomplete responses, with systematic drop-off occurring in the **later questions** of the survey.

The research assistant hypothesizes that attention difficulties make it harder for some teens to complete longer questionnaires, leading to systematic missingness that depends on an **observed characteristic** rather than the missing answers themselves.

**Your Investigation:**

1. **Explore** the missing data pattern across ADHD severity levels  
2. **Diagnose** the likely missingness mechanism (MCAR / MAR / MNAR)  
3. **Apply** a remedy and evaluate its impact on the data



In [ ]:
# Case 2: set-up & quick EDA - Extended 10-question survey
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 12 participants with varying ADHD severity and progressive survey fatigue
case2 = pd.DataFrame({
    "participant_id": [101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112],
    "adhd_score":     [ 3,   7,  10,  12,  14,  15,  16,  17,  17,  18,   5,   9],  # DSM 0-18 scale
    
    # Questions 1-7: Everyone completes these (regardless of ADHD)
    "attitude_q1":    [ 2,   3,   4,   3,   5,   2,   4,   5,   3,   4,   2,   3],
    "attitude_q2":    [ 3,   2,   3,   4,   4,   3,   5,   4,   2,   5,   3,   2],
    "attitude_q3":    [ 1,   2,   2,   3,   3,   4,   3,   2,   4,   3,   2,   1],
    "attitude_q4":    [ 2,   1,   3,   2,   4,   3,   4,   3,   2,   4,   1,   2],
    "attitude_q5":    [ 3,   2,   4,   3,   2,   4,   3,   4,   2,   3,   3,   2],
    "attitude_q6":    [ 2,   3,   2,   4,   3,   2,   2,   3,   4,   2,   2,   3],
    "attitude_q7":    [ 4,   2,   3,   2,   4,   1,   3,   2,   1,   4,   3,   2],
    
    # Later questions (Q8-Q10): Only ADHD ≥16 participants drop out
    "attitude_q8":    [ 1,   3,   2,   3,   4,   2, np.nan, np.nan, np.nan, np.nan, 2, 3],
    "attitude_q9":    [ 3,   2,   1,   2,   3,   4, np.nan, np.nan, np.nan, np.nan, 1, 2],
    "attitude_q10":   [ 2,   4,   3,   1,   2,   3,   2, np.nan, np.nan, np.nan, 3, 1]
})

display(case2)

print("\nMissing count by column:")
print(case2.isna().sum())

print("\nMissing count by participant:")
attitude_cols = [f"attitude_q{i}" for i in range(1, 11)]
missing_by_participant = case2[attitude_cols].isna().sum(axis=1)
print(missing_by_participant)

# Create comprehensive analysis
case2['total_missing'] = case2[attitude_cols].isna().sum(axis=1)
case2['completion_rate'] = (10 - case2['total_missing']) / 10 * 100

print("\nCompletion Analysis:")
completion_summary = case2[['participant_id', 'adhd_score', 'total_missing', 'completion_rate']].copy()
print(completion_summary)

# Visualization
print("\nMissing Data Pattern Analysis:")
plt.figure(figsize=(8, 6))
plt.scatter(case2["adhd_score"], case2["total_missing"], alpha=0.7, s=80, c='steelblue')
plt.xlabel("ADHD Score (0-18 scale)")
plt.ylabel("Number of Missing Items")
plt.title("ADHD Severity vs. Missing Items")
plt.axvline(x=16, color='red', linestyle='--', alpha=0.7, label='Clinical threshold (≥16)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

# Summary statistics
print(f"\nADHD Score Analysis:")
complete_cases = case2[case2['total_missing']==0]
incomplete_cases = case2[case2['total_missing']>0]

print(f"Mean ADHD score - Complete cases (0 missing): {complete_cases['adhd_score'].mean():.1f}")
print(f"Mean ADHD score - Incomplete cases (1+ missing): {incomplete_cases['adhd_score'].mean():.1f}")
print(f"\nSeverity breakdown:")
print(f"Participants with ADHD ≥16: {len(case2[case2['adhd_score']>=16])}")
print(f"Mean completion rate for ADHD ≥16: {case2[case2['adhd_score']>=16]['completion_rate'].mean():.1f}%")
print(f"Mean completion rate for ADHD <16: {case2[case2['adhd_score']<16]['completion_rate'].mean():.1f}%")

### 1️⃣ Diagnose the Missingness Mechanism  
*Write your reasoning below.*

- **Observed pattern:** Only participants with **higher ADHD scores** drop questions near the end.  
- **Likely mechanism:**  
- **Justification (why MAR fits better than MCAR or MNAR):**  


### 2️⃣ Listwise Deletion Analysis – What Do We Lose?
Before applying imputation, let's first see what happens with **listwise deletion** - the "default" approach that simply removes incomplete cases.

**Key Questions:**
- Which participants get excluded?
- How does this change our sample characteristics?
- What bias might this introduce for MAR data?

> This will help us understand **why** imputation might be necessary for this MAR scenario.

In [ ]:
# --- Step 1: Listwise Deletion Analysis ---
attitude_cols = [f"attitude_q{i}" for i in range(1, 11)]
complete_cases = case2.dropna(subset=attitude_cols)

print(f"Original sample: {len(case2)} participants")
print(f"Complete cases: {len(complete_cases)} participants") 
print(f"Lost: {len(case2) - len(complete_cases)} participants")

print(f"\nWho gets excluded:")
excluded_participants = case2[~case2.index.isin(complete_cases.index)]
print(excluded_participants[['participant_id', 'adhd_score', 'total_missing']])

print(f"\nMean ADHD score - Original: {case2['adhd_score'].mean():.1f}")
print(f"Mean ADHD score - Complete cases: {complete_cases['adhd_score'].mean():.1f}")
print(f"High-ADHD (≥16) lost: {len(case2[case2['adhd_score']>=16]) - len(complete_cases[complete_cases['adhd_score']>=16])}")

### 3️⃣ Imputation Comparison – Preserve Individual Patterns?
Now let's compare **two imputation approaches** to see which better handles this MAR scenario:

| Option | Approach | Best For |
|--------|----------|----------|
| **A. Sample Mean Imputation** | Use population averages | Assumes missing participants are "typical" |
| **B. Participant-Specific Mean**  | Use each person's own pattern | Preserves individual differences |

**Task:** Try both approaches and compare how they handle high-ADHD participants differently.

In [ ]:
# --- Step 2: Compare Imputation Approaches ---
print("🔄 IMPUTATION COMPARISON")
print("=" * 50)

# ----- Option A: Sample Mean Imputation -----
clean_sample = case2.fillna(case2.mean(numeric_only=True))

# ----- Option B: Participant-Specific Mean Imputation -----
clean_individual = case2.copy()
attitude_cols = [f"attitude_q{i}" for i in range(1, 11)]

for idx, row in clean_individual.iterrows():
    participant_responses = row[attitude_cols]
    participant_mean = participant_responses.mean()  # excludes NaN automatically
    clean_individual.loc[idx, attitude_cols] = participant_responses.fillna(participant_mean)

# Compare the two approaches for high-ADHD participant (ID 110)
id110_original = case2[case2["participant_id"]==110][attitude_cols].iloc[0]
id110_sample = clean_sample[clean_sample["participant_id"]==110][attitude_cols].iloc[0]
id110_individual = clean_individual[clean_individual["participant_id"]==110][attitude_cols].iloc[0]

print(f"\n📊 Example: ID 110 (ADHD=18)")
print(f"Personal mean from completed questions: {id110_original.mean():.2f}")
print(f"Sample approach - Imputed values: {id110_sample[['attitude_q8', 'attitude_q9', 'attitude_q10']].values}")
print(f"Individual approach - Imputed values: {id110_individual[['attitude_q8', 'attitude_q9', 'attitude_q10']].values}")

print(f"\n📈 Overall Question Means After Imputation:")
print("Sample Mean Approach:")
print(clean_sample[attitude_cols].mean().round(2))
print("\nParticipant-Specific Approach:")
print(clean_individual[attitude_cols].mean().round(2))

### 3️⃣ Reflection Questions
#### **Answer Reflection Questions in Working Table 1 in the Group Document**

**Answer based on your chosen method:**

#### If you chose **Participant-Specific Mean Imputation**:
- How do the imputed values differ from sample means?
- Does this better preserve individual response patterns?
- What happens when a participant has very few completed questions?

#### If you chose **Sample Mean Imputation**:
- Do all high-ADHD participants get identical imputed values?
- Does this ignore their individual response patterns?
- How might this bias correlations with ADHD scores?

**Write 3-4 sentences comparing participant-specific vs. sample-wide imputation for MAR data.**

## Case 3 – "The Treatment Drop-Out Study"  
A substance abuse treatment program tracks adolescent progress over 6 months using self-report questionnaires about **current alcohol use** and **treatment motivation**. 

**The Pattern:** During data collection, researchers notice that participants who started with **higher baseline alcohol use** and **lower treatment motivation** systematically fail to complete follow-up surveys. These participants become non-responsive and cannot be reached for their 6-month assessments.

**The Dilemma:** Is this missing data due to practical issues (harder to contact high-risk teens) or because participants who relapsed don't want to report their current drinking levels?

**Your Investigation:**
1. **Explore** the pattern of baseline characteristics vs. follow-up completion
2. **Diagnose** the likely missingness mechanism (MCAR / MAR / MNAR)  
3. **Apply** a remedy and evaluate its impact on the data

In [ ]:
# Case 3: Treatment study - Missing data pattern
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

case3 = pd.DataFrame({
    "participant_id": [101, 101, 102, 102, 103, 103, 201, 201, 202, 202, 203, 203],
    "timepoint":      ["baseline", "6-month", "baseline", "6-month", "baseline", "6-month", 
                       "baseline", "6-month", "baseline", "6-month", "baseline", "6-month"],
    
    # Baseline risk factors predict who will drop out
    "alcohol_days":   [15, np.nan, 12, np.nan, 10, np.nan,  # High-risk participants missing
                        3,       2,  5,       3,  2,       1], # Low-risk participants continue
    "motivation":     [ 2, np.nan,  3, np.nan,  3, np.nan,  # High baseline use → dropout
                        8,       9,  7,       8,  9,       9]  # High motivation → continue
})

# Ensure proper ordering
case3 = case3.sort_values(["participant_id", "timepoint"]).reset_index(drop=True)
display(case3)

print("\nBaseline Risk vs. Follow-up Completion:")
baseline = case3[case3["timepoint"] == "baseline"].reset_index(drop=True)
followup_counts = case3.groupby("participant_id")["alcohol_days"].count()

# Create completion analysis
completion_analysis = pd.DataFrame({
    "participant_id": baseline["participant_id"],
    "baseline_alcohol_days": baseline["alcohol_days"],
    "baseline_motivation": baseline["motivation"],
    "completed_followup": [followup_counts[pid] > 1 for pid in baseline["participant_id"]]
})
print(completion_analysis)

# Pattern analysis
completers = completion_analysis[completion_analysis["completed_followup"]]
dropouts = completion_analysis[~completion_analysis["completed_followup"]]

print(f"\nPattern Evidence:")
print(f"Mean baseline alcohol days - Completers: {completers['baseline_alcohol_days'].mean():.1f}")
print(f"Mean baseline alcohol days - Drop-outs: {dropouts['baseline_alcohol_days'].mean():.1f}")
print(f"Mean baseline motivation - Completers: {completers['baseline_motivation'].mean():.1f}")
print(f"Mean baseline motivation - Drop-outs: {dropouts['baseline_motivation'].mean():.1f}")

# Box plot visualization
print("\n" + "="*50)
print("📊 VISUAL ANALYSIS")
print("="*50)

plt.figure(figsize=(8, 6))
completion_data = [
    completers['baseline_alcohol_days'].values,
    dropouts['baseline_alcohol_days'].values
]
box_plot = plt.boxplot(completion_data, labels=['Completed', 'Dropped Out'], patch_artist=True)
box_plot['boxes'][0].set_facecolor('lightgreen')
box_plot['boxes'][1].set_facecolor('lightcoral')
plt.ylabel('Baseline Alcohol Days/Month')
plt.title('Alcohol Use Distribution by Follow-up Completion Status')
plt.grid(True, alpha=0.3)
plt.show()

# Summary statistics table
print(f"\n📈 COMPLETION SUMMARY TABLE:")
print("-" * 40)
summary_table = pd.DataFrame({
    'Group': ['Completers', 'Drop-outs', 'Difference'],
    'N': [len(completers), len(dropouts), ''],
    'Mean Alcohol Days': [f"{completers['baseline_alcohol_days'].mean():.1f}", 
                         f"{dropouts['baseline_alcohol_days'].mean():.1f}",
                         f"{dropouts['baseline_alcohol_days'].mean() - completers['baseline_alcohol_days'].mean():.1f}"],
    'Mean Motivation': [f"{completers['baseline_motivation'].mean():.1f}", 
                       f"{dropouts['baseline_motivation'].mean():.1f}",
                       f"{dropouts['baseline_motivation'].mean() - completers['baseline_motivation'].mean():.1f}"]
})
print(summary_table.to_string(index=False))


### 1️⃣ Diagnose the Missingness Mechanism  
*Analyze the pattern and determine the mechanism.*

- **Observed pattern:** Participants with highest baseline alcohol use and lowest motivation are missing follow-up data
- **Likely mechanism (MCAR / MAR / MNAR):**  
- **Justification:** Why does this pattern suggest your chosen mechanism?  
- **Key consideration:** Does the missingness depend on observed baseline characteristics, unobserved follow-up values, or random factors?

**Think carefully:** Are high-risk participants missing because:
- **(MAR)** They're harder to reach due to their baseline characteristics (lifestyle, contact info changes)?
- **(MNAR)** They may have relapsed and don't want to report their current alcohol use levels?

### 2️⃣ Choose & Apply a Remedy  
Select **one** approach for handling the missing follow-up data:

| Option | Approach | When It Makes Sense |
|--------|----------|-------------------|
| **A. Listwise Deletion** | Remove incomplete participants | Assumes missing data won't bias results |
| **B. LOCF (Last Observation Carried Forward)** | Use baseline values for missing follow-up | Assumes no change over time |
| **C. Mean Imputation by Timepoint** | Use average follow-up values | Assumes missing participants are "typical" |

> **Task:** Choose the approach that best fits your diagnosed mechanism and evaluate its assumptions.

In [ ]:
# --- Remedy code cell ---
# ⬇️ Uncomment the approach you decide to test.

# ----- Option A: Listwise Deletion -----
# clean = case3.dropna()

# ----- Option B: LOCF (Last Observation Carried Forward) -----
# clean = (
#     case3
#     .sort_values(["participant_id", "timepoint"])
#     .groupby("participant_id")
#     .ffill()  # forward-fill within participant
# )

# ----- Option C: Mean Imputation by Timepoint -----
# clean = case3.copy()
# followup_means = case3[case3["timepoint"] == "6-month"].mean(numeric_only=True)
# mask = (clean["timepoint"] == "6-month")
# clean.loc[mask, "alcohol_days"] = clean.loc[mask, "alcohol_days"].fillna(followup_means["alcohol_days"])
# clean.loc[mask, "motivation"] = clean.loc[mask, "motivation"].fillna(followup_means["motivation"])

print("Data after chosen remedy:")
display(clean)

# Compare trajectories
print(f"\nMean alcohol days by timepoint:")
print("BEFORE imputation (complete cases only):")
print(case3.groupby("timepoint")["alcohol_days"].mean())
print("\nAFTER chosen remedy:")
print(clean.groupby("timepoint")["alcohol_days"].mean())

### 3️⃣ Reflection – Evaluate Your Remedy  
#### **Answer Reflection Questions in Working Table 1 in the Group Document**

**Analyze the consequences of your chosen approach:**

- **Method used:** [A/B/C]  
- **Assumption made:** What does this method assume about the missing data?  
- **Bias check:** How did the remedy change the trajectory from baseline to follow-up?  
- **Validity concern:** Does your method's assumption match your diagnosed mechanism?  

Write 3-4 sentences evaluating whether your chosen remedy creates systematic bias and how it might affect conclusions about treatment effectiveness.

## Case 4 – "The Mental Health Screening Study"  
A university counseling center conducts an annual mental health survey asking students about **sleep patterns**, **academic stress**, **social support**, and **depression symptoms**. The final section asks: **"In the past 2 weeks, how often have you felt down, depressed, or hopeless?"** (0=Not at all, 1=Several days, 2=More than half the days, 3=Nearly every day)

**The Missing Data Problem:** Several students with **high stress levels**, **poor sleep**, and **low social support** left the depression questions blank. However, the research team has strong predictors available: students' **stress scores**, **sleep quality**, and **social connectedness** - all factors known to strongly correlate with depression symptoms.

**Your Investigation:**
1. **Explore** the pattern of missing data vs. predictor variables
2. **Diagnose** the likely missingness mechanism (MCAR / MAR / MNAR)  
3. **Choose the best remedy** given the available predictors and mechanism

In [ ]:
# Case 4: Mental health survey with strong predictors
import pandas as pd, numpy as np

case4 = pd.DataFrame({
    "student_id":      [101, 102, 103, 104, 105, 106, 107, 108, 109, 110],
    "stress_level":    [ 2,   8,   9,   3,   7,   4,   9,   5,   8,   6],  # 1-10 scale
    "sleep_quality":   [ 8,   3,   2,   7,   4,   6,   2,   7,   3,   5],  # 1-10 scale (higher=better)
    "social_support":  [ 8,   2,   1,   7,   3,   6,   1,   6,   2,   4],  # 1-10 scale (higher=better)
    "semester_gpa":    [3.6, 2.1, 1.8, 3.4, 2.5, 3.2, 1.9, 3.0, 2.2, 2.8], # current semester GPA
    
    # Depression screening question: Missing for high-risk students
    "depression_score": [ 0,   2, np.nan, 0, np.nan, 1, np.nan, 0, np.nan, 1]  # 0-3 scale, NaN=refused
})

display(case4)

print("\nMissing data pattern:")
print(f"Missing responses: {case4['depression_score'].isna().sum()} out of {len(case4)} students")

print("\nWho refused to answer:")
missing_students = case4[case4['depression_score'].isna()]
print(missing_students[['student_id', 'stress_level', 'sleep_quality', 'social_support']])

print("\nPredictor patterns:")
complete_cases = case4.dropna()
missing_cases = case4[case4['depression_score'].isna()]

print(f"Mean stress - Complete answers: {complete_cases['stress_level'].mean():.1f}")
print(f"Mean stress - Missing answers: {missing_cases['stress_level'].mean():.1f}")
print(f"Mean sleep quality - Complete answers: {complete_cases['sleep_quality'].mean():.1f}")
print(f"Mean sleep quality - Missing answers: {missing_cases['sleep_quality'].mean():.1f}")
print(f"Mean social support - Complete answers: {complete_cases['social_support'].mean():.1f}")
print(f"Mean social support - Missing answers: {missing_cases['social_support'].mean():.1f}")

### 1️⃣ Diagnose the Missingness Mechanism  
*Analyze the pattern and determine the mechanism.*

- **Observed pattern:** Students with highest stress, poorest sleep, and lowest social support refused to answer
- **Likely mechanism (MCAR / MAR / MNAR):**  
- **Justification:** Does the missingness depend on observed characteristics, unobserved values, or random factors?
- **Key insight:** What makes this different from purely random missing data?

### 2️⃣ Choose & Apply a Remedy  
Given the available **strong predictors** (stress, sleep, social support), select the best approach:

| Option | Approach | When It Makes Sense |
|--------|----------|-------------------|
| **A. Listwise Deletion** | Remove students with missing data | Assumes missing data won't bias results |
| **B. Mean Imputation** | Use average depression score for missing | Assumes missing students are "typical" |
| **C. Regression Imputation** ★ | Predict using stress, sleep, social factors | **Uses available predictor information** |

> **Task:** Choose the method that best leverages the strong predictors available in this dataset.

In [ ]:
# --- Remedy code cell ---
# ⬇️ Uncomment the approach you decide to test.

# ----- Option A: Listwise Deletion -----
# clean = case4.dropna()
# print("Data after listwise deletion:")
# print(f"Sample size: {len(clean)} students")
# print(f"Mean depression score: {clean['depression_score'].mean():.2f}")

# ----- Option B: Mean Imputation -----
# clean = case4.copy()
# depression_mean = case4['depression_score'].mean()  # Calculate from non-missing
# clean['depression_score'] = clean['depression_score'].fillna(depression_mean)
# print("Data after mean imputation:")
# print(f"Imputed value: {depression_mean:.2f}")
# print(f"Overall depression mean: {clean['depression_score'].mean():.2f}")

# ----- Option C: Regression Imputation -----
from sklearn.linear_model import LinearRegression
import numpy as np

clean = case4.copy()
complete = case4.dropna()

# Fit model on complete cases
predictors = ['stress_level', 'sleep_quality', 'social_support', 'semester_gpa']
X_train = complete[predictors]
y_train = complete['depression_score']

model = LinearRegression().fit(X_train, y_train)

# Predict for missing cases
missing_idx = case4['depression_score'].isna()
X_missing = case4.loc[missing_idx, predictors]
predictions = model.predict(X_missing)

# Apply predictions (round to nearest valid score: 0, 1, 2, 3)
predictions_rounded = np.clip(np.round(predictions), 0, 3)
clean.loc[missing_idx, 'depression_score'] = predictions_rounded
clean.loc[missing_idx, 'imputed'] = True
clean['imputed'] = clean['imputed'].fillna(False)

print("Data after regression imputation:")
print(f"Predicted depression scores for missing students: {predictions_rounded}")
print(f"Model R²: {model.score(X_train, y_train):.3f}")

display(clean)

# Compare approaches
print(f"\nDepression score comparison:")
print(f"Complete cases only: {case4['depression_score'].mean():.2f}")
print(f"After chosen remedy: {clean['depression_score'].mean():.2f}")

# Show model coefficients
print(f"\nModel insights:")
for pred, coef in zip(predictors, model.coef_):
    print(f"{pred}: {coef:.3f}")

### 3️⃣ Reflection – Evaluate Your Remedy  
#### **Answer Reflection Questions in Working Table 1 in the Group Document**

**Analyze the consequences of your chosen approach:**

- **Method used:** [A/B/C]  
- **Predictor utilization:** How well did your method use the available stress/sleep/social support information?
- **Bias check:** How did your remedy change the overall depression score estimate?  
- **Validity concern:** Does your method's assumption match your diagnosed mechanism?

Write 3-4 sentences evaluating: Why is regression imputation particularly well-suited for this mental health scenario? What are the risks of ignoring the strong predictor relationships available in counseling center data?

## Exercise 3 – Part 2  ·  Missing Data in ABCD Substance Use Environments & Attitudes Surveys ( Case Studies 5 – 8 )

You and your partners will tackle another four mini-investigations, each built around a different ABCD survey instrument measuring substance use environments and attitudes and a distinctive missing-data wrinkle:

| Case | Theme & Instrument | Missing-Data Focus |
|------|--------------------|--------------------|
| **5** | *Youth Cigarette Expectancies* (`su_y_cigexp`) | Wave-specific “not-administered” blanks + 777 Declines |
| **6** | *Parental Rules* for alcohol & vaping (`su_p_rule`) | Wave design blanks • Branch-skip blanks • 777 Declines |
| **7** | *Household MJ-Smoke Exposure* (`su_p_des__mj__smoke`) | Numeric follow-ups with separate **DK flags 999** |
| **8** | *School Attendance & Grades* (`fc_p_sag`) | Ordinal codes 444 / 777 vs. branch logic & IEP skip |
---
### Team Assignment Table

| Group Member | Case Study |
|--------------|-----------------|
| **Member 1** | Case 5 – |
| **Member 2** | Case 6 – |
| **Member 3** | Case 7 – |
| **Member 4** | Case 8 – |

### What you’ll do 🔍
Each notebook follows an identical roadmap:

1. **Setup** – import libraries & load the supplied *synthetic* CSV.  
2. **Codebook Audit** – enumerate unique codes and fill in a *missing-count* challenge.  
3. **Visual Diagnostics** – plot a `missingno` matrix and inspect crosstabs to separate design skips, branch skips, DKs, and true gaps.  
4. **Naïve vs Clean Metric** – compute a quick statistic two ways to show bias.  
5. **Reflection Slides** – answer three short prompts in your group PowerPoint.

Along the way you’ll see **_fill-in-the-blank lines_** (marked `___` or `# TODO`).  
Use the Copilot/ChatGPT hints under each challenge if you get stuck.

---

### Deliverables 📑
After you complete your case study, then complete **Working Table 2** in your shared PowerPoint.


## Cast Study 5

In **Case Study 5** we will explore youth **Cigarette Expectancies (ASCQ)** variables from the ABCD Study.

Dataset highlights  
* **Instrument**: `su_y_cigexp` – thoughts, feelings, and beliefs about smoking  
* **Key item**: `su_y_cigexp_001`  “During the day, smoking can help kill time if there is nothing to do.”  
  * Added part-way through 2-year follow-up (Wave 2) ► not administered at Wave 1  
* Response codes (ordinal): `1 Never … 5 Always`, `0 Uncertain`, plus special `777 Decline`, blanks = missing  

In this case study you will  
1. Audit the coding structure for the ASCQ items  
2. Visualise and diagnose wave-specific missingness  
3. Compare naïve vs. cleaned means for a positive-expectancy score  
4. Reflect on coding, mechanism, and cleaning choices  

📚 Helpful references:  
* [ABCD Docs – su_y_cigexp](https://docs.abcdstudy.org/latest/documentation/non_imaging/su.html#su_y_cigexp)  
* [ABCD Data Exploration Portal](https://abcd.deapscience.com/#/my-datasets/create-dataset)


In [ ]:
# Step 1 Code Cell: imports and data load
import pandas as pd, numpy as np, matplotlib.pyplot as plt, missingno as msno
np.random.seed(42)

df = pd.read_csv("synthetic_su_y_cigexp_sample.csv")  # 20 × 2 = 40 rows
df.head()


### Step 2: List every unique code—including `0/1–5`, `777`, blanks—for six ASCQ items.

**Target variables**

* `su_y_cigexp_001`          (new positive item)  
* `su_y_cigexp__neg_001`    “Smoking will make a person cough.”  
* `su_y_cigexp__neg_002`    “Smoking makes people look ridiculous or silly.”  
* `su_y_cigexp__pos_001`    “Cigarettes help with concentration.”  
* `su_y_cigexp__pos_002`    “When someone is sad, smoking helps them feel better.”  
* `su_y_cigexp__pos_003`    “The look and feel of a cigarette in the mouth is good.”  

---

#### Code Challenge #1 – Count Missing Values  
Fill in the blank so `missing_counts` shows the **number of NaN** entries per column.

**Copilot Prompts for Understanding**

**1. How can you create a boolean mask to identify missing (NaN) values in a pandas DataFrame or Series?**
- Use the `.isna()` method to generate a boolean mask where `True` indicates missing values.

**2. How can you summarise the number of `True` values in a boolean mask?**
- Apply the `.sum()` method to count the `True` values in the mask.



In [ ]:
# Step 1 Code Cell
vars_to_audit = [
    "su_y_cigexp_001",
    "su_y_cigexp__neg_001",
    "su_y_cigexp__neg_002",
    "su_y_cigexp__pos_001",
    "su_y_cigexp__pos_002",
    "su_y_cigexp__pos_003",
]

# --- Code Challenge answer placeholder
missing_counts = df[vars_to_audit]. _____ # Fill in the missing code here
print(missing_counts)

# Build tidy code-frequency table
records = []
for var in vars_to_audit:
    vc = df[var].value_counts(dropna=False).sort_index()
    for code, cnt in vc.items():
        records.append({"variable": var, "code": code, "count": int(cnt)})

codebook_df = pd.DataFrame(records)
codebook_df.head()


### Step 3: Plot a heat-map to spot:  
* *Design* missingness – Wave 1 blanks for `su_y_cigexp_001`  
* *Item* non-response – 777 Declines or Uncertain 0s


In [ ]:
cols = vars_to_audit
msno.matrix(df[cols], figsize=(9, 4))
plt.title("Missingness Matrix – ASCQ Items")
plt.show()


Verify that **all** Wave 1 rows are missing for `su_y_cigexp_001`, while Wave 2 rows contain real data or 777/0.


In [ ]:
pd.crosstab(df["session_id"], df["su_y_cigexp_001"].isna(),
            rownames=["Wave"], colnames=["Missing?"])


### Step 4 · Naïve vs. Clean Mean Bias (LO #3)

**Prompt:** Compare the mean of a simple *Positive Expectancy Index* = mean of  
`su_y_cigexp_001`, `__pos_001`, `__pos_002`, `__pos_003`  

* **Naïve** analyst: converts all blanks / 777 to 0 ("Never").  
* **Clean** analyst: treats 777 & blanks as `NaN` and excludes them from the mean.

**Goal:** Demonstrate how improper handling of missing data codes can bias substantive findings by comparing two approaches to computing a cigarette expectancy score.

**Steps:**
1. Create a "naïve" index by replacing all missing values and 777 codes with 0
2. Create a "clean" index by properly converting 777 codes to `NaN` and excluding them
3. Compare the means and interpret the bias direction and magnitude
4. Reflect on implications for research conclusions about adolescent smoking attitudes

In [ ]:
pos_items = ["su_y_cigexp_001", "su_y_cigexp__pos_001",
             "su_y_cigexp__pos_002", "su_y_cigexp__pos_003"]

# Clean copy
df_clean = df.copy()
df_clean[pos_items] = df_clean[pos_items].replace({777: np.nan})

# Naïve index (zeros for everything missing/777)
naive_index = df[pos_items].replace({np.nan: 0, 777: 0}).astype(float).mean(axis=1)
naive_mean  = naive_index.mean()

# Clean index (exclude NaN)
clean_index = df_clean[pos_items].astype(float).mean(axis=1, skipna=True)
clean_mean  = clean_index.mean()

pct_diff = (naive_mean - clean_mean) / clean_mean * 100
print(f"Naïve mean = {naive_mean:.2f}")
print(f"Clean mean = {clean_mean:.2f}")
print(f"Percent difference = {pct_diff:.1f}%")


### Reflection Questions  
**Answer Reflection Questions in Working Table 2 in the Group Document**

Write about 3 sentences per question.

**Q1 (LO 1)** – For one ASCQ item, decode blank / 0 / 777 and state which indicates non-response vs. true uncertainty.  

**Q2 (LO 2)** – Compare Wave 1 “design” missingness (item not administered) to 777 Declines. Which mechanism fits each? Explain.  

**Q3 (LO 3)** – Using Step 4, report naïve vs. clean means & sample sizes. Why does zero-filling bias the index, and how does proper cleaning improve it?


## Case Study 6

**Case Study 6** explores how ABCD encodes **parental rules** for alcohol and e-cigarette/vape use.

Key design facts  
* **Instrument:** `su_p_rule` – parental approval / household rules.  
* **Gate (rules) items:**  
  * `su_p_rule__alc_001` – rules about adolescent **alcohol** drinking (asked at every wave)  
  * `su_p_rule__nic__vape_001` – rules about adolescent **ENDS/vaping** (added at the 3-year visit ⇒ *not administered* at Wave-1)  
* **Follow-ups:**  
  * Alcohol: `__alc_001__01` (same rules for all?) & `__alc_001__02` (penalties?)  
  * ENDS:   `__nic__vape_001__01` (same rules for all?)  
* Codes: ordinal levels (1–6), plus `777  Decline`, blanks = missing.

What you’ll do  
1. Audit code frequencies & missing codes  
2. Visualise wave- and branch-related missingness  
3. Check branch logic for follow-ups  
4. Quantify bias from naïve vs. cleaned strict-rule indexing  
5. Reflect on 777 vs. branch skips vs. not-administered blanks

📚 Docs: [**ABCD Docs: su_p_rule**](https://docs.abcdstudy.org/latest/documentation/non_imaging/su.html#su_p_rule)   |   [Data Explorer](https://abcd.deapscience.com/#/my-datasets/create-dataset)


In [ ]:
# Step 0 Code Cell: imports & load
import pandas as pd, numpy as np, matplotlib.pyplot as plt, missingno as msno
np.random.seed(42)

df = pd.read_csv("synthetic_su_p_rule_sample.csv")   # 40 rows (20×2)
df.head()


**Goal:** List every unique code—including `777` and blanks—for the six variables below.

* `su_p_rule__alc_001`
* `su_p_rule__alc_001__01`
* `su_p_rule__alc_001__02`
* `su_p_rule__nic__vape_001`
* `su_p_rule__nic__vape_001__01`

---

### Code Challenge: Count Missing Values  
Fill in the blank so `missing_counts` tallies `NaN`s per variable.

**Copilot Prompts for Understanding:**

1. **How do you identify missing values in a pandas DataFrame?**  
   - Use the `.isna()` method to create a boolean mask where `True` indicates missing values.

2. **How can you count missing values for each column in a DataFrame?**  
   - Chain `.sum()` after `.isna()` to count `True` values (missing entries) for each column.


In [ ]:
# Step 1 Code Cell
vars_to_audit = [
    "su_p_rule__alc_001",
    "su_p_rule__alc_001__01",
    "su_p_rule__alc_001__02",
    "su_p_rule__nic__vape_001",
    "su_p_rule__nic__vape_001__01",
]

# Challenge answer placeholder
missing_counts = df[vars_to_audit]._______ # Fill in
print(missing_counts)

# tidy codebook table
records = []
for v in vars_to_audit:
    for code, n in df[v].value_counts(dropna=False).sort_index().items():
        records.append({"variable": v, "code": code, "count": int(n)})
codebook_df = pd.DataFrame(records)
codebook_df.head()


### Step 2: Heatmap of Missing Values
We expect three distinct missingness sources:

1. **Wave design:** ENDS items blank at Wave-1  
2. **Branch logic:** follow-ups blank when gate rule is missing or `777`  
3. **True non-response:** `777  Decline`

Visualise them.


In [ ]:
msno.matrix(df[vars_to_audit + ["session_id"]], figsize=(9,4))
plt.title("Missingness – Parental Rules Items")
plt.show()


### Step 3 · Branch-Logic Crosstabs 

Branch-logic crosstabs help verify whether follow-up questions are properly skipped when gate conditions aren't met, revealing the difference between intentional design skips and true non-response.

**Code Challenge #2 – Fill-in Crosstab**  
Complete the blank so each table shows counts of missing vs. present follow-ups across gate codes.

**Copilot Prompts for Understanding:**

1. **How do you create a boolean mask to identify missing (NaN) values in a pandas Series?**
   - Use the `.isna()` method to create a boolean mask where `True` indicates missing values.

2. **What does `pd.crosstab()` show when you cross-tabulate a gate variable with missing status?**
   - It reveals the relationship between gate responses and whether follow-up questions are missing, helping verify branch logic patterns.


In [ ]:
pairs = [("su_p_rule__alc_001",        "su_p_rule__alc_001__01"),
         ("su_p_rule__alc_001",        "su_p_rule__alc_001__02"),
         ("su_p_rule__nic__vape_001",  "su_p_rule__nic__vape_001__01")]

for gate, follow in pairs:
    ctab = pd.crosstab(df[gate],
                       df[follow].____,       # Fill in
                       rownames=[gate], colnames=[f"{follow} missing?"])
    display(ctab)

## Step 4 · Naïve vs. Clean “Strict-Rule” Index (LO #3)

Create a binary **Strict-Rule** variable:

* Alcohol strict = 1 if `su_p_rule__alc_001 == 1` (no drinking ever) else 0  
* Naïve analyst: treats blanks & 777 as *non-strict* (0)  
* Clean analyst: sets blanks/777 to `NaN` and drops them

Compare prevalence.


In [ ]:
strict_alc = (df["su_p_rule__alc_001"] == 1).astype(float)   # NaN where not 1/else?
strict_alc_naive = strict_alc.fillna(0)
naive_prop  = strict_alc_naive.mean()

strict_alc_clean = strict_alc.copy()
strict_alc_clean[df["su_p_rule__alc_001"].isin([777]) | df["su_p_rule__alc_001"].isna()] = np.nan
clean_prop  = strict_alc_clean.mean()

pct_bias = (naive_prop - clean_prop) / clean_prop * 100
print(f"Naïve strict-rule prevalence = {naive_prop:.2%}")
print(f"Clean strict-rule prevalence = {clean_prop:.2%}")
print(f"Bias = {pct_bias:+.1f}%  (positive = overestimate)")


### Reflection Questions  
**Answer Reflection Questions in Working Table 2 in the Group Document**

Write about 3 sentences per question.

**Q1 (LO 1):** For a selected variable, explain what each code represents—blanks for branch‐skip, 777 for non‐response, and levels 1–6 for valid answers.

**Q2 (LO 2)** – Using heat-map & crosstabs, contrast wave-design blanks (ENDS Wave-1) with branch blanks (follow-ups) and 777. Assign MCAR / MAR / MNAR.

**Q3 (LO 3)** – Report naïve vs. clean strict-rule prevalence & bias. Explain why zero-filling mis-represents household strictness and how excluding 777/branch blanks fixes it.


# Case Study 7
### Case Study 7 examines parent-reported *marijuana exposure* in the home.

Key facts  
* **Gate item** `su_p_des__mj__smoke_001`  
  * 0 = No, 1 = Yes, 777 = Refused  
* **Numeric follow-ups** (asked only if gate == 1)  
  * `…__01`  Days per week smoke occurs  (0–7)  
  * `…__02`  Hours per day exposed      (1–12)  
* **DK flags** `…__01__dk`, `…__02__dk` (999 = Don’t-know)  

> **Heads-up:**  (`555 not-administered`, `888 branch-skip`, `999 don’t-know`, …) are **defined only for *categorical* ABCD variables**.  
> The two follow-up items in this case are *quantitative* (counts of days & hours).  
> That means the REDCap form stores **blank cells (`NaN`)** instead of sentinel codes, and relies on a *separate DK-flag column* to capture a “Don’t-know” response.

In this case you will  
1. Audit codes for gate & flag variables  
2. Parse blanks vs. DK vs. true missing in numeric columns  
3. Visualise patterns of design skips, DK, and unexpected gaps  
4. Compare naïve vs. cleaned mean exposure hours  
5. Reflect on how numeric follow-ups differ from coded categorical items

📚 **Helpful references**:
- ABCD documentation for this measure: [ABCD Docs: su_p_des](https://docs.abcdstudy.org/latest/documentation/non_imaging/su.html#su_p_des)  
- To look up actual variables and definitions, use the [ABCD Data Exploration Portal](https://abcd.deapscience.com/#/my-datasets/create-dataset)


In [ ]:
# Step 0 Code Cell – imports & data load
import pandas as pd, numpy as np, matplotlib.pyplot as plt, missingno as msno
np.random.seed(42)

df = pd.read_csv("synthetic_mj_smoke_sample.csv")   # 20 × 2 = 40 rows
df.head()


### Step 1 · Codebook Audit
**Goal:** List every unique code across the five columns.

* `su_p_des__mj__smoke_001`
* `su_p_des__mj__smoke_001__01`
* `su_p_des__mj__smoke_001__01__dk`
* `su_p_des__mj__smoke_001__02`
* `su_p_des__mj__smoke_001__02__dk`

---

### Code Challenge: Count Missing versus DK  
Fill in the blank so `missing_and_dk` shows, for each **numeric follow-up** variable, how many rows are *blank* (`NaN`) **and** how many are flagged DK (`==999`) separately.

**Copilot Prompts for Understanding:**

1. **How do you identify missing values in a pandas DataFrame?**
   - Use the `.isna()` method to create a boolean mask where `True` indicates missing values.

2. **How can you count missing values for each column in a DataFrame?**
   - Chain `.sum()` after `.isna()` to count `True` values (missing entries) for each column.

In [ ]:
# Step 1 Code Cell
vars_gate   = ["su_p_des__mj__smoke_001"]
vars_num    = ["su_p_des__mj__smoke_001__01", "su_p_des__mj__smoke_001__02"]
vars_dk     = [c + "__dk" for c in vars_num]

# quick counts
missing_and_dk = (df[______.to_frame("blank") # Fill in
                  .join((df[vars_dk] == 999).sum().rename("dk")))
print(missing_and_dk, "\n")

# tidy frequency table for gate
print(df["su_p_des__mj__smoke_001"].value_counts(dropna=False).sort_index())

### Step 2 · Plot the five variables

Visualize: 
* Wave-1 design blanks (numeric follow-ups not asked when gate == 0/NaN)  
* DK blanks (value missing but DK flag == 999)  
* Any odd gaps that need investigation


In [ ]:
msno.matrix(df[vars_gate + vars_num + vars_dk], figsize=(9,4))
plt.title("Missingness – Marijuana-Smoke Exposure items")
plt.show()


### Step 3 · Branch-Logic Verification
We expect numeric follow-ups to be **present** only when gate == 1. Run the code below to ensure this.


In [ ]:
for num in vars_num:
    print(f"\n{num}")
    display(pd.crosstab(df["su_p_des__mj__smoke_001"],
                        df[num].notna(),
                        rownames=["Gate"], colnames=["present?"]))


### Optional Step 3b · Merge Numeric + DK Flag and Visualize  
Combine each numeric follow-up with its DK flag so you can treat “DK” as an explicit category when plotting.

**New variable**  
`hours_exposure_combined` =  

| If | Then |  
|---|---|  
| `…__02__dk == 999` | string `"DK"` |  
| numeric value present | that value (int) |  
| else | `NaN` (branch skip / not asked) |

After creating the combined column, make a histogram (numeric bars) plus a single bar for `"DK"`.


In [ ]:
# Combine Hours + DK flag into one column
hours_num = df["su_p_des__mj__smoke_001__02"]
hours_dk  = df["su_p_des__mj__smoke_001__02__dk"]

hours_combined = hours_num.copy()
hours_combined[hours_dk == 999] = "DK"   # string label
df["hours_exposure_combined"] = hours_combined

# Quick visualization: numeric bars + DK bar
fig, ax = plt.subplots(figsize=(6,4))
# separate numeric and DK
numeric_vals = pd.to_numeric(hours_combined, errors="coerce").dropna()
numeric_vals.hist(ax=ax, bins=range(1,14), edgecolor="black")
# add DK count
dk_count = (hours_combined == "DK").sum()
ax.bar(x=13, height=dk_count, width=0.8, color="gray", label="DK")
ax.set_xticks(list(range(1,13)) + [13])
ax.set_xticklabels(list(range(1,13)) + ["DK"])
ax.set_xlabel("Hours per day (or DK)")
ax.set_ylabel("Count")
ax.set_title("Exposure Hours (numeric) vs. DK responses")
ax.legend()
plt.show()


### Step 4 · Naïve vs Clean Mean Exposure Hours
Create **HoursExposure** = numeric column `…__02`.

* **Naïve** analyst → sets blanks/DK→0 hours  
* **Clean** analyst → blanks where gate !=1 or DK flag ==999 are `NaN`

Compare means.


In [ ]:
hours_raw   = df["su_p_des__mj__smoke_001__02"].copy()

# make a cleaned version
hours_clean = hours_raw.copy()
# remove DKs
hours_clean[df["su_p_des__mj__smoke_001__02__dk"] == 999] = np.nan
# remove branch skips (gate != 1)
hours_clean[df["su_p_des__mj__smoke_001"] != 1] = np.nan

naive_mean = hours_raw.fillna(0).mean()
clean_mean = hours_clean.mean()
bias       = (naive_mean - clean_mean) / clean_mean * 100

print(f"Naïve mean = {naive_mean:.2f} hrs")
print(f"Clean mean = {clean_mean:.2f} hrs")
print(f"Bias = {bias:+.1f}%  (pos = over-estimate)")


### Reflection Questions  
**Answer Reflection Questions in Working Table 2 in the Group Document**

Write about 3 sentences per question.

**Q1 (LO 1)** – Decode blank, DK flag 999, and numeric values 0-12.  
Which blanks are design skips, DK, or data glitches?

**Q2 (LO 2)** – Using the heat-map & crosstabs, identify MCAR / MAR / MNAR patterns across the five variables.

**Q3 (LO 3)** – Summarise naïve vs. clean mean HoursExposure and bias. Explain why filling blanks with 0 distorts average exposure.


## Case Study 8

In **Case Study 8**, we will work with a synthetic version of the **School Attendance & Grades [Parent]** dataset (`fc_p_sag`) for 14-year-old participants.  
This dataset replicates how the ABCD Study structures and codes data for parent-reported school experiences, including:

- **Ordinal grade categories** (coded 1–12)
- **Special values** like `444 = Not Applicable`, `777 = Decline to answer`, and blank cells for skipped questions
- **Branch logic**, where follow-up questions are only asked if a parent first reports their child has an IEP (Individualized Education Plan)

In this case study, you will:
1. Audit the coding structure for key variables  
2. Visualize and diagnose missingness patterns  
3. Apply branch logic checks and develop a cleaning strategy

---

📚 **Helpful references**:
- ABCD documentation for this measure: [ABCD Docs – fc_p_sag](https://docs.abcdstudy.org/latest/documentation/non_imaging/fc.html#fc_p_sag)  
- To look up actual variables and definitions, use the [ABCD Data Exploration Portal](https://abcd.deapscience.com/#/my-datasets/create-dataset)


In [ ]:
# Step 0 Code Cell: imports and data load

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import missingno as msno

# Reproducibility
np.random.seed(42)

# Load the synthetic dataset for Case Study 4
df = pd.read_csv("synthetic_fc_p_sag_sample.csv")

# Preview the first 5 rows
df.head()


### Step 1 · Codebook Audit (LO #1)

**Goal:** Enumerate and inspect **all unique codes**—including `0/1`, `444` (Not Applicable), `777` (Decline), `999` (Don’t Know), and blanks—for six key variables. This will help you map each code back to ABCD’s standardized conventions and prepare a tidy lookup table.

**Target variables:**
- `fc_p_sag_004`  
- `fc_p_sag_005`  
- `fc_p_sag_006a`  
- `fc_p_sag_006a__01`  
- `fc_p_sag_006a__02___2`  

Run `value_counts(dropna=False)` on each to capture blanks as well as special codes.

### Code Challenge: Counting Missing Values

Fill in the blank so that `missing_counts` becomes a pandas `Series` showing the number of `NaN` (missing) entries for each audited variable:

#### Copilot Prompt Suggestions

**1. Which pandas DataFrame method helps you identify missing (NaN) values across columns?**
- Use the `.isna()` method to create a boolean mask where `True` indicates missing values.

**2. How can you aggregate a boolean mask of missing entries to count them per column?**
- Apply `.sum()` on the boolean mask to count `True` values (missing entries) for



In [ ]:
# Step 1 Code Cell: enumerate codes and counts for each target variable

# List of variables to audit
vars_to_audit = [
    "fc_p_sag_004",
    "fc_p_sag_005",
    "fc_p_sag_006a",
    "fc_p_sag_006a__01",
    "fc_p_sag_006a__02___2"
]

# Code Challenge: Display missing counts for each variable
missing_counts = df[vars_to_audit].___________
print(missing_counts)

# Prepare an empty list to collect codebook entries
records = []

# Loop through each variable, get counts (including NaN), and record them
for var in vars_to_audit:
    vc = df[var].value_counts(dropna=False).sort_index()
    total = len(df)
    for code, cnt in vc.items():
        records.append({
            "variable": var,
            "code":      code,
            "count":     int(cnt),
            "percent":   cnt / total * 100
        })

# Build a tidy summary DataFrame
codebook_df = pd.DataFrame(records)

# Display for review
codebook_df.sort_values(["variable", "code"]).reset_index(drop=True)


### Step 3 · Missingness Heat-Map 
Visualizing missing data patterns helps you quickly identify where and how data are absent—revealing branch-logic skips, item nonresponse, and potential systematic gaps. We’ll use the **missingno** library to generate a heat‐map matrix showing present vs. missing values for our key GPA and IEP variables.


In [ ]:
# Step 3 Code Cell: plot missingness matrix

import missingno as msno
import matplotlib.pyplot as plt

# Select the four variables of interest
cols = [
    "fc_p_sag_004",
    "fc_p_sag_005",
    "fc_p_sag_006a",
    "fc_p_sag_006a__01",
    "fc_p_sag_006a__02___2"
]

# Plot the missingness matrix
msno.matrix(df[cols], figsize=(8, 4), fontsize=12)
plt.title("Missingness Heat-Map for fc_p_sag Variables")
plt.show()


### Step 4 · Branch-Logic Crosstab

When a parent reports **no IEP** (`0`) or **Don’t Know** (`999`), the follow-up items (`fc_p_sag_006a__01`, `fc_p_sag_006a__02___2`) should be blank (missing).  
The crosstab below shows, for each IEP response, how many follow-up entries are missing (`True`) vs. present (`False`), confirming the branch logic.

### Code Challenge: Branch-Logic Crosstab Fill-in-the-Blank

Complete the blank so that `ctab` cross-tabulates **IEP status** against whether each follow-up column is missing:

**Copilot Prompt Suggestions**

1. Which pandas Series method gives you a boolean mask of missing (NaN) values?
- Use the `.isna()` method to identify missing values in a pandas Series.

2. How does `pd.crosstab` treat True/False when summarizing two series?
- `pd.crosstab` treats `True` and `False` as distinct categories, allowing you to summarize their counts across two


In [ ]:
# Step 4 Code Cell: verify branch-logic skip patterns

followups = ["fc_p_sag_006a__01", "fc_p_sag_006a__02___2"]

for col in followups:
    ctab = pd.crosstab(
        df["fc_p_sag_006a"],
        df[col].___, # Fill in the blank
        rownames=["IEP Status"],
        colnames=[f"{col} Missing?"]
    )
    print(f"\nMissingness crosstab for `{col}`:")
    display(ctab)


### Step 5 · Naïve vs. Clean Mean Bias Demonstration

**Prompt:** Simulate how a naïve analyst who replaces **all** missing and special codes (`NaN`, `444`, `777`) with `0` in `fc_p_sag_004` (unexcused absences) would bias the mean estimate.  

1. Compute the “naïve” mean after zero‐replacement.  
2. Retrieve the properly cleaned mean from `df_clean`.  
3. Calculate the percent difference.  
4. Comment briefly on whether the naïve approach under- or over-estimates the true mean.


In [ ]:
# Step 5 Code Cell: compare naïve vs. cleaned mean for fc_p_sag_004

import numpy as np

# First, create a properly cleaned dataset
df_clean = df.copy()

# Proper cleaning: convert special codes to NaN, keep only valid numeric responses
df_clean["fc_p_sag_004"] = df_clean["fc_p_sag_004"].replace({444: np.nan, 777: np.nan})

# 1. Naïve mean: replace NaN, 444, 777 → 0
naive_series = df["fc_p_sag_004"].replace({np.nan: 0, 444: 0, 777: 0}).astype(float)
naive_mean   = naive_series.mean()

# 2. Clean mean: properly handle special codes by excluding them
clean_mean   = df_clean["fc_p_sag_004"].mean()  # This automatically excludes NaN values

# 3. Percent difference
pct_diff     = (naive_mean - clean_mean) / clean_mean * 100

# 4. Print results
print(f"Naïve mean = {naive_mean:.2f}")
print(f"Clean mean = {clean_mean:.2f}")
print(f"Percent difference = {pct_diff:.1f}%")

# 5. Show sample sizes for context
print(f"\nSample sizes:")
print(f"Naïve approach (includes all data): {len(naive_series)} participants")
print(f"Clean approach (excludes special codes): {df_clean['fc_p_sag_004'].count()} participants")

# 6. Interpretation
if naive_mean < clean_mean:
    print(f"\n📉 The naïve approach UNDERESTIMATES the true mean by {abs(pct_diff):.1f}%")
else:
    print(f"\n📈 The naïve approach OVERESTIMATES the true mean by {pct_diff:.1f}%")

### Reflection Questions  
**Answer Reflection Questions in Working Table 2 in the Group Document**

Write about 3 sentences per question.

**Q1 (LO 1)** – For one variable, decode blank / 444 / 777 / 999 and state which indicates logical skip vs. true non-response.

**Q2 (LO 2)** – Using heat-map & crosstabs, contrast structured codes (444/777/999) with NaN blanks. Assign MCAR / MAR / MNAR mechanisms and explain why.

**Q3 (LO 3)** – Report naïve vs. clean means & sample sizes for `fc_p_sag_004`. Explain why zero-filling biases the mean and how proper cleaning fixes it.